### Import relevant libraries and files

In [ ]:
from pynq import allocate
from pynq import Overlay
import numpy as np
import pynq.lib.dma
import time
import pynq
import pandas as pd

In [ ]:
#import weights and bias from trained model and import test data
# weight_0 = np.load('weight_0.npy')
test = np.loadtxt("dataset/test.csv")
data_label = pd.read_csv("dataset/test.csv")['Activity']

### FPGA

In [ ]:
# load bitstream inside FPGA
bitstream = Overlay('bitstream.bit')  # dummy file name
# dma module
dma = overlay.axi_dma_0  

In [ ]:
# loading weights and bias into neurons
for i in range(?):
    for k in range(?):
        input_buffer0[k] = weight_0[k][i]
    dma.sendchannel.transfer(input_buffer0)
    dma.sendchannel.wait()

In [ ]:
# fpga calculation

### Ultra96

In [ ]:
# evaluation using cpu

### FPGA Performance

In [ ]:
# record time taken for fpga to predict and present as graph

### CPU Performance

In [ ]:
# record time taken for cpu to predict and present as graph

### Speed and Accuracy